In [ ]:
from google.colab import drive
import os

# 1. Mount Drive
drive.mount('/content/drive')

# 2. Clone NHÁNH convnext-v2-dev
if not os.path.exists('/content/FR_Photometric_Stereo'):
    !git clone -b convnext-v2-dev https://github.com/minh4923/FR_Photometric_Stereo.git

# 3. Di chuyển vào thư mục dự án
%cd /content/FR_Photometric_Stereo

# 4. Cài đặt thư viện
!pip install -q albumentations==1.3.1 timm


Mounted at /content/drive
Cloning into 'FR_Photometric_Stereo'...
remote: Enumerating objects: 142, done.
remote: Counting objects: 100% (142/142), done.
remote: Compressing objects: 100% (99/99), done.
remote: Total 142 (delta 48), reused 132 (delta 38), pack-reused 0 (from 0)
Receiving objects: 100% (142/142), 10.79 MiB | 32.51 MiB/s, done.
Resolving deltas: 100% (48/48), done.
/content/FR_Photometric_Stereo
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.7/125.7 kB 8.0 MB/s eta 0:00:00


In [ ]:
%cd /content/FR_Photometric_Stereo
import warnings
warnings.filterwarnings("ignore")
import torch
from torch.optim import Adam
from torch.optim.lr_scheduler import CosineAnnealingWarmRestarts
import albumentations as A
import os
import pandas as pd

from going_modular.dataloader.multitask import create_multitask_datafetcher
from going_modular.model.MTLFaceRecognition import MTLFaceRecognition
from going_modular.loss.MultiTaskLoss import MultiTaskLoss
from going_modular.train_eval.train import fit
from going_modular.utils.transforms import RandomResizedCropRect, GaussianNoise
from going_modular.utils.MultiMetricEarlyStopping import MultiMetricEarlyStopping
from going_modular.utils.ModelCheckPoint import ModelCheckpoint
from going_modular.utils.ExperimentManager import ExperimentManager

# --- CẤU HÌNH ---
device = "cuda" if torch.cuda.is_available() else "cpu"
seed = 42
torch.manual_seed(seed)

EXPERIMENT_NAME = "Single_ALBEDO_Shuffle"

CONFIGURATION = {
    'note': EXPERIMENT_NAME,
    'dataset_dir': '/content/drive/MyDrive/Photometric_DB_Full/',
    'type': 'albedo',


    'use_sampler': False,  # <--- False: Random Shuffle | True: PK Sampler


    'device': device,
    'epochs': 120,
    'batch_size': 32,
    'image_size': 112,
    'base_lr': 1e-4,
    'backbone': 'miresnet18',
    'num_classes': None,

    # Trọng số
    'loss_gender_weight': 1.0,      'loss_da_gender_weight': 0.1,
    'loss_emotion_weight': 0.5,     'loss_da_emotion_weight': 0.1,
    'loss_pose_weight': 1.0,        'loss_da_pose_weight': 0.1,
    'loss_spectacles_weight': 0.5,  'loss_da_spectacles_weight': 0.1,
    'loss_facial_hair_weight': 0.5, 'loss_da_facial_hair_weight': 0.1,
}

# --- 1. MANAGER ---
manager = ExperimentManager(CONFIGURATION)
manager.log_text(f"TRAINING MODE: {CONFIGURATION['type']} | Backbone: {CONFIGURATION['backbone']}")
manager.log_text(f"SAMPLER MODE: {'PK Sampler' if CONFIGURATION['use_sampler'] else 'Random Shuffle'}")

# --- 2. DATASET ---
train_csv_path = os.path.join(CONFIGURATION['dataset_dir'], 'dataset/train_split.csv')
if not os.path.exists(train_csv_path): train_csv_path = os.path.join(CONFIGURATION['dataset_dir'], 'train_split.csv')

df_temp = pd.read_csv(train_csv_path)
CONFIGURATION['num_classes'] = df_temp['id'].max() + 1

train_transform = A.Compose([
    RandomResizedCropRect(CONFIGURATION['image_size']),
    GaussianNoise(p=0.2), A.HorizontalFlip(p=0.5)
], additional_targets={'albedo': 'image', 'depthmap': 'image'})

test_transform = A.Compose([A.Resize(CONFIGURATION['image_size'], CONFIGURATION['image_size'])],
                           additional_targets={'albedo': 'image', 'depthmap': 'image'})


train_dl, test_dl, _ = create_multitask_datafetcher(CONFIGURATION, train_transform, test_transform)

# --- 3. MODEL & LOSS ---
model = MTLFaceRecognition(CONFIGURATION['backbone'], CONFIGURATION['num_classes'])
criterion = MultiTaskLoss(train_csv_path, CONFIGURATION)
optimizer = Adam(model.parameters(), lr=CONFIGURATION['base_lr'])
scheduler = CosineAnnealingWarmRestarts(optimizer, T_0=20, T_mult=2)

# --- 4. CHECKPOINT ---
ckpt_saver = ModelCheckpoint(
    output_dir=manager.ckpt_dir,
    mode='max',
    best_metric_name='auc_id_cosine'
)

early_stopping = MultiMetricEarlyStopping(
    monitor_keys=['cosine_auc'], patience=1500, mode='max', verbose=0,
    save_dir=manager.ckpt_dir, start_from_epoch=0
)

# --- 5. RUN ---
fit(
    conf=CONFIGURATION,
    start_epoch=0,
    model=model,
    train_dataloader=train_dl,
    test_dataloader=test_dl,
    criterion=criterion,
    optimizer=optimizer,
    scheduler=scheduler,
    early_stopping=early_stopping,
    model_checkpoint=ckpt_saver,
    existing_manager=manager
)

Kết quả truyền trực tuyến bị cắt bớt đến 5000 dòng cuối.
├─────────────────────┼───────────┼─────────────────────┤
│ loss_gender         │  0.18133  │ 0.25557011200322044 │
├─────────────────────┼───────────┼─────────────────────┤
│ loss_da_gender      │  0.449221 │ 0.5113267203172048  │
├─────────────────────┼───────────┼─────────────────────┤
│ loss_emotion        │  0.532579 │ 0.6278200149536133  │
├─────────────────────┼───────────┼─────────────────────┤
│ loss_da_emotion     │  0.643653 │ 0.6693776581022475  │
├─────────────────────┼───────────┼─────────────────────┤
│ loss_pose           │  0.50063  │ 0.6010327041149139  │
├─────────────────────┼───────────┼─────────────────────┤
│ loss_da_pose        │  0.614158 │ 0.6860544217957391  │
├─────────────────────┼───────────┼─────────────────────┤
│ loss_facial_hair    │  0.273975 │ 0.46903888053364223 │
├─────────────────────┼───────────┼─────────────────────┤
│ loss_da_facial_hair │  0.505495 │ 0.5882918702231513  │
├──────────────